In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# See database_helpers.py
run database_helpers.py

In [ ]:
# First, since I know python can handle it, we'll gather all the dimensions' data
dim_year=select_dicts("""
SELECT "Id" as "YearId", "Year" FROM "Dim_Year"
""")
dim_year=pd.DataFrame(dim_year, columns=list(dim_year[0].keys()))

dim_eez=select_dicts("""
SELECT "Id" as "EEZId", "EEZ", "HDICountryName" FROM "Dim_EEZ"
""")
dim_eez=pd.DataFrame(dim_eez, columns=list(dim_eez[0].keys()))

dim_taxon=select_dicts("""
SELECT "Id" as "TaxonId", "Species" FROM "Dim_Taxon"
""")
dim_taxon=pd.DataFrame(dim_taxon, columns=list(dim_taxon[0].keys()))

dim_functional_group=select_dicts("""
SELECT "Id" as "FunctionalGroupId", "FunctionalGroupName" FROM "Dim_Functional_Group"
""")
dim_functional_group=pd.DataFrame(dim_functional_group, columns=list(dim_functional_group[0].keys()))

dim_commercial_group=select_dicts("""
SELECT "Id" as "CommercialGroupId", "CommercialGroupName" FROM "Dim_Commercial_Group"
""")
dim_commercial_group=pd.DataFrame(dim_commercial_group, columns=list(dim_commercial_group[0].keys()))

dim_fishing_sector=select_dicts("""
SELECT "Id" as "FishingSectorId", "FishingSectorName" FROM "Dim_Fishing_Sector"
""")
dim_fishing_sector=pd.DataFrame(dim_fishing_sector, columns=list(dim_fishing_sector[0].keys()))

dim_gear=select_dicts("""
SELECT "Id" as "GearId", "GearName" FROM "Dim_Gear"
""")
dim_gear=pd.DataFrame(dim_gear, columns=list(dim_gear[0].keys()))

dim_end_use_type=select_dicts("""
SELECT "Id" as "EndUseTypeId", "EndUseTypeName" FROM "Dim_End_Use_Type"
""")
dim_end_use_type=pd.DataFrame(dim_end_use_type, columns=list(dim_end_use_type[0].keys()))

dim_hdi=select_dicts("""
SELECT "Id" as "HDIId", "HDICountryName" FROM "Dim_HDI_Current"
""")
dim_hdi=pd.DataFrame(dim_hdi, columns=list(dim_hdi[0].keys()))

dim_hdi_historical=select_dicts("""
SELECT "Id" as "HDIIdHistorical", "HDICountryName", "HDIYear" FROM "Dim_HDI"
""")
dim_hdi_historical=pd.DataFrame(dim_hdi_historical, columns=list(dim_hdi_historical[0].keys()))

In [ ]:
# In order to break up processing, we're going to chunk this data by year
all_years = select("""
SELECT "Year" FROM "Dim_Year"
""")

all_years = [year[0] for year in all_years]

In [ ]:
# Since we'll be doing this in chunks, we'll have to manually keep track of this as we go
current_max_id=0

# Get these special values. Since we rely on merges later, it will be easy enough
# to backfill this as a replacement value for NaNs after the respective merges.
unknown_taxon = dim_taxon.loc[dim_taxon["Species"] == "Unidentified Taxon", "TaxonId"][0]
unknown_gear = dim_gear.loc[dim_gear["GearName"] == "Unknown", "GearId"][0]

for year in all_years:
    print("Processing data for the year {}".format(year))
    # Get a chunk of data
    data_chunk = select("""
    SELECT
        "source_file",
        "year"::INTEGER AS "Year",
        "scientific_name" AS "Species",
        "functional_group" AS "FunctionalGroupName",
        "commercial_group" AS "CommercialGroupName",
        "fishing_sector" AS "FishingSectorName",
        "gear_type" AS "GearName",
        "end_use_type" AS "EndUseTypeName",
        "tonnes" AS "CatchWeight",
        "landed_value" AS "ExVesselValue"
    FROM "Raw_Fact"
    WHERE "year" = (%s)::VARCHAR
    """, [year])
    data_chunk = pd.DataFrame(
        data_chunk,
        columns=[
            "source_file",
            "Year",
            "Species",
            "FunctionalGroupName",
            "CommercialGroupName",
            "FishingSectorName",
            "GearName",
            "EndUseTypeName",
            "CatchWeight",
            "ExVesselValue"
        ]
    )

    # The EEZ Value is parsed out of the filename
    data_chunk["EEZ"] = data_chunk["source_file"].str.replace(r"\D", "", regex=True)
    data_chunk["EEZ"] = data_chunk["EEZ"].astype("int")
    data_chunk = data_chunk.merge(dim_eez, how="left")

    # replace values for all dims with ids using crafty merges
    data_chunk = data_chunk.merge(dim_year, how="left")

    data_chunk = data_chunk.merge(dim_taxon, how="left")
    data_chunk.loc[data_chunk["TaxonId"].isna(), "TaxonId"] = unknown_taxon

    data_chunk.loc[:,"GearName"] = data_chunk["GearName"].str.title()
    data_chunk = data_chunk.merge(dim_gear, how="left")
    data_chunk.loc[data_chunk["GearId"].isna(), "GearId"] = unknown_gear

    data_chunk = data_chunk.merge(dim_functional_group, how="left")

    data_chunk = data_chunk.merge(dim_commercial_group, how="left")

    data_chunk = data_chunk.merge(dim_fishing_sector, how="left")

    data_chunk = data_chunk.merge(dim_end_use_type, how="left")

    data_chunk = data_chunk.merge(dim_hdi, how="left")

    # Joining the historical hdi data gets a little trickier
    # Now that we've got our Dim_Year.YearId merged in, we can 
    # rename some columns and do a compound merge.
    data_chunk.rename(columns={"Year": "HDIYear"}, inplace=True)
    data_chunk = data_chunk.merge(dim_hdi_historical, how="left")
    
    # Strip to just the columns we want in our fact table so the rest of
    # processing has less to work with
    data_chunk = data_chunk[[
        "YearId",
        "EEZId",
        "TaxonId",
        "FunctionalGroupId",
        "CommercialGroupId",
        "FishingSectorId",
        "GearId",
        "EndUseTypeId",
        "HDIId",
        "HDIIdHistorical",
        "CatchWeight",
        "ExVesselValue"
    ]]
    
    # Convert column types
    data_chunk = data_chunk.astype({
        "CatchWeight": "float",
        "ExVesselValue": "float"
    })
    
    
    # Convert the dollar values to whole dollars and cents
    data_chunk["ExVesselValue"] = data_chunk["ExVesselValue"] * 1000000
    data_chunk = data_chunk.round({"ExVesselValue": 2})

    # Create an id
    data_chunk.reset_index(inplace=True)
    data_chunk.rename(columns={"index":"Id"}, inplace=True)
    data_chunk["Id"] = data_chunk["Id"] + current_max_id
    current_max_id = current_max_id + len(data_chunk.index)

    # Replace NaN with None so that we get sql NULLs
    data_chunk.replace({np.nan:None}, inplace=True)

    print(". . . inserting into database.")
    # bulk insert
    bulk_insert("""
    INSERT INTO "Fact_Catch"
    (
        "Id",
        "YearId",
        "EEZId",
        "TaxonId",
        "FunctionalGroupId",
        "CommercialGroupId",
        "FishingSectorId",
        "GearId",
        "EndUseTypeId",
        "HDIId",
        "HDIIdHistorical",
        "CatchWeight",
        "ExVesselValue"
    )
    VALUES
    %s
    """, data_chunk)
    print("Done.")
    print("\n\n")